## Assignment 1 - IO

Author: David Henning (worked with Natasha Watkins, Chandni Raja and Jonathan Kowarski)

In [1]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS, tsa
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy.optimize import minimize
from scipy import ndimage

In [2]:
d = pd.read_csv("PS1_Data/OTC_Data.csv", sep='\t')
#d[d.duplicated(subset=['store', 'week', 'brand'], keep=False)] # No duplicates at store-week-brand level

In [3]:
d_demographic = pd.read_csv("PS1_Data/OTCDemographics.csv", sep='\t')

In [4]:
instruments = pd.read_csv("PS1_Data/OTCDataInstruments.csv", sep='\t')

In [5]:
d.sort_values(by=['store', 'week', 'brand'], inplace=True)

In [6]:
#Try to reproduce summary stat table from the problem set
d['revenue'] = d['sales_'] * d['price_']
d['revenue_brand'] = d.groupby(['brand'])['revenue'].transform('sum')
d['sales_brand'] = d.groupby(['brand'])['sales_'].transform('sum')
d['price_brand'] = d.groupby(['brand'])['price_'].transform('mean')
d['cost_brand'] = d.groupby(['brand'])['cost_'].transform('mean')
d['branded'] = d['brand']
d.loc[d['branded'] <= 9, 'branded'] = 1
d.loc[d['branded'] > 9, 'branded'] = 0
d_summary = d[['brand', 'revenue_brand', 'sales_brand', 'price_brand', 'cost_brand']]
d_summary = d_summary[:11]
d_summary['revenue_tot'] = d_summary['revenue_brand'].sum()
d_summary['share'] = d_summary['revenue_brand'] / d_summary['revenue_tot'] * 0.62

In [7]:
d_instruments = pd.read_csv("PS1_Data/OTCDataInstruments.csv", sep='\t')

### Problem 1

Consider utility function for good $j$ in store-week $t$ for consumer $i$:
    $$ u_{ijt} = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \epsilon_{jt} $$
   

###### (1) Estimate this model using OLS with price and promotion as product characteristics

In [8]:
# calculate the shares
df = d[['store', 'week', 'brand', 'prom_', 'price_', 'cost_', 'sales_']].copy()
df['sales_store_week'] = df.groupby(['store', 'week'])['sales_'].transform('sum')
df['s'] = df['sales_'] / df['sales_store_week'] * 0.62 # All of the brands have a total market share of 62%

In [9]:
# Transform variables for regression, recall outside options share is 0.38
s_0 = 0.38
df['δ'] = np.log(df['s']) - np.log(s_0)
df['constant'] = 1

In [10]:
# Run OLS regression
x = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
y = df['δ']
results1 = OLS(y, x).fit()
model1_α = results1.params['price_']
#print(results2.summary())

###### (2) Estimate this model using OLS with price and promotion as product characteristics and brand dummies

In [11]:
#Add dummies to dataframe
b_dummies = pd.get_dummies(df['brand'])
df_dummies = df.merge(b_dummies, left_index = True, right_index = True)

In [12]:
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
y = df_dummies['δ']
results2 = OLS(y, x).fit()
model2_α = results2.params['price_']
#print(results2.summary())

###### (3) Estimate this model using OLS with price and promotion as product characteristics and store-brand dummies

In [13]:
store_brand = df['store'].astype('str') + '_' + df['brand'].astype('str') 
bs_dummies = pd.get_dummies(store_brand)
df_bsdummies = df.merge(bs_dummies, left_index = True, right_index = True)

In [14]:
x = df_bsdummies.drop(['δ', 'store', 's', 'brand', 'week', 'cost_', 'sales_', 'sales_store_week'], 1)
y = df_bsdummies['δ']

In [15]:
results3 = OLS(y, x).fit()
model3_α = results3.params['price_']
#print(results3.summary())

###### (4) Estimate the models of 1, 2 and 3 using wholesale cost as an instrument.

In [16]:
# Run IV regression for first problem
x = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
z = df.drop(['δ', 'store', 'week', 'brand', 'sales_', 'price_', 'sales_store_week', 's',], 1)
y = df['δ']
results4 = IV2SLS(y, x, instrument=z).fit()
#print(results4.summary())

In [17]:
# Run IV regression for second problem
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
z = df_dummies.drop(['δ', 'store', 'week', 'brand', 'sales_', 'price_', 'sales_store_week', 's',], 1)
y = df['δ']
results4 = IV2SLS(y, x, z).fit()
#print(results4.summary())

In [18]:
# Run IV regression for third problem
x = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
z = df_bsdummies.drop(['δ', 'store', 'week', 'brand', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df['δ']
results4 = IV2SLS(y, x, z).fit()
#print(results4.summary())

###### (5) Estimate the models of 1, 2 and 3 using the Hausman instrument (average price in other markets).

In [19]:
#Calculate hausman instrument, and add to all the datasets
grouped = df.groupby(['brand', 'week'])
n = grouped['price_'].transform('count')
mean = grouped['price_'].transform('mean')
df['price_iv'] = (mean*n - df['price_'])/(n-1)
df_dummies['price_iv'] = (mean*n - df['price_'])/(n-1)
df_bsdummies['price_iv'] = (mean*n - df['price_'])/(n-1)
df.sort_values(by=['brand', 'week', 'store'], inplace=True)

In [20]:
#Run the IV with  the Hausman instrument for (1)
x = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_iv', 'sales_', 'sales_store_week', 's',], 1)
z = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df['δ']
results5 = IV2SLS(y, x, z).fit()
#print(results5.summary())

In [21]:
#Run the IV with  the Hausman instrument for (2)
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_iv', 'sales_', 'sales_store_week', 's',], 1)
z = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df_dummies['δ']
results5 = IV2SLS(y, x, z).fit()
#print(results5.summary())

In [22]:
#Run the IV with  the Hausman instrument for (3)
x = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_iv', 'sales_', 'sales_store_week', 's',], 1)
z = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df_bsdummies['δ']
results5 = IV2SLS(y, x, z).fit()
#print(results5.summary())

###### (6) Using the analytic formula for elasticity of the logit model, compute the mean own-price elasticities for all brand in the market using the estimates in 1, 2 and 3. Do these results make sense?

In [23]:
d_summary['model1_η'] = -model1_α * d_summary['price_brand'] * (1 - d_summary['share'])
d_summary['model2_η'] = -model2_α * d_summary['price_brand'] * (1 - d_summary['share'])
d_summary['model3_η'] = -model3_α * d_summary['price_brand'] * (1 - d_summary['share'])
#d_summary

## Problem 2

Consider utility function for good $j$ in store-week $t$ for consumer $i$:
    $$ u_{ijt} = X_{jt}\beta + \beta_{ib}B_{jt}\text{(Branded Product)} + \alpha_i p_{jt} + \xi_{jt} + \epsilon_{ijt} $$
Where:
$$\beta_{ib} = \sigma_B\nu_i$$
$$\alpha_i = \alpha + \sigma_I I_i$$

Hence $$ u_{ijt} = X_{jt}\beta + \sigma_B\nu_iB_{jt}\text{(Branded Product)} + (\alpha + \sigma_I I_i) p_{jt} + \xi_{jt} + \epsilon_{ijt} $$

Or, rearranging $$ u_{ijt} = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \sigma_B\nu_iB_{jt}\text{(Branded Product)} + \sigma_I I_i p_{jt} + \epsilon_{ijt} $$


In [24]:
d_blp = pd.merge(d, d_demographic, on=['store', 'week'])
d_blp['constant'] = 1
d_blp['sales_store_week'] = d_blp.groupby(['store', 'week'])['sales_'].transform('sum')
d_blp['s'] = d_blp['sales_'] / d_blp['sales_store_week'] * 0.62 # All of the brands have a total market share of 62%
d_blp['market'] = d_blp.groupby(['store', 'week']).ngroup()
d_blp.sort_values(by=['store', 'week', 'brand'], inplace=True)

In [25]:
#tryout
def iterate_shares2(β, σ_B, σ_I, δ, p, v, B, I, X, ids, s):
    max_iter = 100
    dist = 10e3      # Initial distance
    tol = 10e-10      # Tolerance
    i = 0

    s_hat = np.zeros(I.shape[0])
        
    for i in range(ns):
        d_blp['numerator'] = np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)
        denominator = d_blp.groupby(['store', 'week'])['numerator'].transform('sum')
        s_hat += d_blp.numerator / denominator * (1 / ns)
            
    return s_hat

In [26]:
#tryout
def iterate_shares3(β, σ_B, σ_I, δ, p, v, B, I, X, ids, s):
    max_iter = 100
    dist = 10e3      # Initial distance
    tol = 1e-10      # Tolerance
    i = 0

    s_hat = np.zeros(I.shape[0])
        
    for i in range(ns):
        numerator = np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)
        denominator = ndimage.sum(numerator, ids, ids) + 1
        s_hat += numerator / denominator * (1 / ns)
            
    return s_hat

In [27]:
def iterate_shares(β, σ_B, σ_I, δ, p, v, B, I, X, ids, s):
    max_iter = 1000
    dist = 1e3      # Initial distance
    tol = 1e-12      # Tolerance
    i = 0

    while i < max_iter and dist > tol:
        s_hat = np.zeros(I.shape[0])
        
        for i in range(ns):
            numerator = np.exp(δ + σ_B * v[i] * B + σ_I * I[:, i] * p)
            denominator =  ndimage.sum(numerator, ids, ids) + 1
            s_hat += numerator / denominator * (1 / ns)
        
        δ_new = δ + np.log(s) - np.log(s_hat)
        dist = np.linalg.norm(δ_new - δ)
        δ = δ_new
        i += 1
            
    return δ

In [28]:
def gmm(params, p, v, B, I, X, Z, ids, s):
    β = params[:-2]
    σ_B = params[-2]
    σ_I = params[-1]
    
    δ_init = X @ β #Initial δ
    
    δ = iterate_shares(β, σ_B, σ_I, δ_init, p, v, B, I, X, ids, s)
    ξ = δ - X @ β
    Q = Z.T @ ξ @ np.linalg.inv(Z.T @ Z) @ Z.T @ ξ
    print(Q)
    
    return Q

In [50]:
# Put everything into arrays
#X = d_blp[['constant', 'proms_']].copy()
X = d_blp[['price_', 'constant']].copy() #Include price, constant and promotions
brand_dummies = pd.get_dummies(d_blp['brand']) #get brand dummies
X = pd.concat([X, brand_dummies], axis=1) # add brand dummies to X
X = X.values #turn it into an array
p = d_blp['price_'].values
B = d_blp['branded'].values
I = d_blp.iloc[:, d_blp.columns.str.startswith('hhincome')].values
Z = instruments.iloc[:, 3:].values
ids = d_blp['market'].values
s = d_blp['s'].values

#Setting the frequencies
ns = I.shape[1]

#Starting values for parameters
np.random.seed(1)
v = np.random.normal(size=ns)

In [51]:
# params = np.ones(X.shape[1] + 2) * 0.1
# params[0] = 0.1
# params[1] = 0.1
# β = params[:-2]
# σ_B = params[-2]
# σ_I = params[-1]
# δ_init = X @ β #Initial δ
# s_hat = iterate_shares2(β, σ_B, σ_I, δ_init, p, v, B, I, X, ids, s)

In [52]:
params = np.ones(X.shape[1] + 2)
params[-2] = 0.1
params[-1] = 0.1
β = params[:-2]
σ_B = params[-2]
σ_I = params[-1]

In [64]:
# params = np.ones(X.shape[1] + 2) * 0.1
# params[0] = 0.1
# params[1] = 0.1
# β = params[:-2]
# σ_B = params[-2]
# σ_I = params[-1]
# δ_init = X @ β #Initial δ
# δ =  iterate_shares(β, σ_B, σ_I, δ_init, p, v, B, I, X, ids, s)
# δ
params = np.ones(X.shape[1] + 2)
params[-2] = 0.1
params[-1] = 0.1

array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       0.1, 0.1])

In [54]:
bounds = [[None, None]] * len(params)
bounds[0] = [None, 0]  # Price coefficient must be negative
# bounds[1] = [0, None]
bounds[-2] = [0, None]
bounds[-1] = [0, None]

In [55]:
#Minimize gmm
params = np.ones(X.shape[1] + 2)
params[-2] = 0.1
params[-1] = 0.1

results2 = minimize(gmm, params, args=(p, v, B, I, X, Z, ids, s), 
                   method='L-BFGS-B',
                   bounds=bounds,
                   tol=1e-4,
                   options={'disp' : True})

3118871.6550044436
3118871.6222178293
3118871.6614650213
3118871.6554787634
3118871.6556505547
3118871.655882247
3118871.655405281
3118871.655693003
3118871.6560720163
3118871.655365304
3118871.6554962015
3118871.6554662567
3118871.6552964924
3118871.65569153
3118871.6542557073
3118872.004810669
673658.4191347624
673658.4040501766
673658.4221427473
673658.4193657376
673658.4194291182
673658.419511507
673658.4193245508
673658.4194566398
673658.4196192288
673658.4193049727
673658.4193695202
673658.4193130296
673658.4192896504
673658.4195054416
673658.4187944299
673658.580019357
16253.70444666126
16253.704458403468
16253.704509486593
16253.704469323382
16253.704439619736
16253.704394873726
16253.704455586978
16253.704454835366
16253.704433206582
16253.70445386326
16253.704461417985
16253.704382495634
16253.704484128819
16253.70454674773
16253.704458023838
16253.70420207011
16249.29823074138
16249.298243346377
16249.298293389531
16249.298253388486
16249.298223683532
16249.298178936364
1624

In [57]:
results.x

array([-8.61115986, -0.87305211,  0.26637237,  0.77839983])

In [58]:
results2.x

array([-8.95676486, -0.94434124,  0.80753453,  0.84225795,  0.90198158,
        0.86552969,  0.78870035,  0.74435416,  0.88121405,  0.82453546,
        1.0501204 ,  0.81673099,  0.53593409,  0.2742315 ,  0.79700168])

In [61]:
results2.x[0] + results2.x[-1] * I.mean()

-0.4885103692995294